In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.metrics import accuracy_score,classification_report,mean_squared_error
import math
from deepctr.models import DeepFM,WDL 
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from tensorflow import keras

In [30]:
def RecommenderDeepFM(train, test, sparse_features, 
                   sequence_features, PK_features, target, batch_size, epochs, 
                   sep=" ",embedding_dim=32,dnn_dropout=0.5,dnn_hidden_units=(128,128), 
                   optimizer='Adam', learning_rate=0.0001):
    print('optimizer:{0} | embedding_dim:{1} | lr:{2} | dropout:{3} | hidden_units:{4} | epochs:{5} | batch_size:{6}'.format(optimizer,embedding_dim,
                                                                            learning_rate,dnn_dropout,dnn_hidden_units,
                                                                            epochs, batch_size))
    traindata = train.copy()
    testdata = test.copy()
    
    def split(x):
        key_ans = x.split(sep)
        key_ans = [k.strip() for k in key_ans]
        for key in key_ans:
            if key not in key2index:
                # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
                key2index[key] = len(key2index) + 1
        return list(map(lambda x: key2index[x], key_ans))  
    
    traindata[target[0]] = pd.to_numeric(traindata[target[0]], downcast='signed')
    
    testdata[target[0]] = pd.to_numeric(testdata[target[0]], downcast='signed')
    
    data = pd.concat([traindata,testdata], ignore_index = True)
    
    # 1.Label Encoding for sparse features,and process sequence features
    for feat in sparse_features:
        lbe = LabelEncoder()
        #data[feat] = lbe.fit_transform(data[feat])  
        data[feat + "_Encoding"] = lbe.fit_transform(data[feat])
    
    fixlen_feature_columns = [SparseFeat(feat + "_Encoding", data[feat + "_Encoding"].nunique(),embedding_dim=embedding_dim)
                            for feat in sparse_features]
    
    
    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns  
    
    for feat in sequence_features:    
        key2index = {}
        templist = list(map(split, data[feat].values))
        length = np.array(list(map(len, templist)))
        max_len = max(length)
        # Notice : padding=`post`
        templist = pad_sequences(templist, maxlen = max_len, padding = 'post', )
        
        use_weighted_sequence = False
        if use_weighted_sequence:
            varlen_feature_columns = [VarLenSparseFeat(SparseFeat(feat + "_Encoding",vocabulary_size=len(key2index) + 1,
                                        embedding_dim = embedding_dim), maxlen = max_len, combiner='mean',weight_name = feat + "_Encoding" + "_weight")]  # Notice : value 0 is for padding for sequence input feature
        else:
            varlen_feature_columns = [VarLenSparseFeat(SparseFeat(feat + "_Encoding",vocabulary_size= len(key2index) + 1,
                                        embedding_dim = embedding_dim), maxlen = max_len, combiner='mean',weight_name = None)]  # Notice : value 0 is for padding for sequence input feature
    
        dnn_feature_columns += varlen_feature_columns
        linear_feature_columns += varlen_feature_columns    
        data[feat + "_Encoding"] = list(templist)  

    # 2. generate input data for model
    encodedfeat = [feat +'_Encoding' for feat in sparse_features + sequence_features]
    encodedfeat = PK_features + target + encodedfeat

    traindata.drop(columns=target,inplace =True)
    trainEncoded = data.merge(traindata,on = PK_features).loc[:,encodedfeat]

    testdata.drop(columns=target,inplace =True)
    testEncoded = data.merge(testdata,on = PK_features).loc[:,encodedfeat]

    model_input = {feat + "_Encoding":trainEncoded[feat + "_Encoding"] for feat in sparse_features} 
    for feat in sequence_features:
        model_input[feat + "_Encoding"] = np.array([d.T for d in trainEncoded[feat + "_Encoding"].values])

    test_model_input = {feat + "_Encoding":testEncoded[feat + "_Encoding"] for feat in sparse_features} 
    for feat in sequence_features:
        test_model_input[feat + "_Encoding"] = np.array([d.T for d in testEncoded[feat + "_Encoding"].values])

    # 3.Define Model,train,predict and evaluate
    DeepFMModel = DeepFM(linear_feature_columns,dnn_feature_columns,task='regression')
    
    opt = ''
    
    if optimizer == 'Adam':        
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        opt = tf.keras.optimizers.SDG(learning_rate=learning_rate)
        
    DeepFMModel.compile("adam", "mse", metrics=['mse'], )
    
    historyDeepFMModel = DeepFMModel.fit(model_input, data[target].values,
                            batch_size=32, epochs=3, verbose=2, validation_split=0.2, )

    # 4. make class predictions with the model and evaluation
    predictions = DeepFMModel.predict(test_model_input)    
    
    maxrating = testEncoded[target].max()[0]
    minrating = testEncoded[target].min()[0]   
    
    predictions_rounded = []
    for x in predictions:
        if x > maxrating:
            predictions_rounded.append(maxrating)
        elif x < minrating:
            predictions_rounded.append(minrating)
        else:
            predictions_rounded.append(round(x[0]))   
    
    MSE = mean_squared_error(testEncoded[target].values, predictions_rounded)
    print("MSE:",MSE)   
    
    return (DeepFMModel, MSE)

In [28]:
traindata = pd.read_csv("./Movies_train.csv") 

In [29]:
testdata = pd.read_csv("./Movies_test.csv") 

In [31]:
sparse_features = ['NAME', 'USER_MD5', 'MOVIE_ID', 
                   'DOUBAN_SCORE', 'DOUBAN_VOTES']

sequence_features = ['ACTORS', 'DIRECTORS', 'GENRES', 'REGIONS', 
                     'TAGS', 'LANGUAGES', 'MovieNameWord']

PK_features = ['USER_MD5', 'MOVIE_ID']

target = ['RATING']

epochs = 3

batch_size = 32

In [32]:
model = RecommenderDeepFM(traindata,testdata, sparse_features, sequence_features,PK_features, target, batch_size, epochs, sep="|")

optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.5 | hidden_units:(128, 128) | epochs:3 | batch_size:32
Epoch 1/3


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


4409/4409 - 218s - loss: 0.9965 - mse: 0.9639 - val_loss: 1.0234 - val_mse: 0.9655
Epoch 2/3
4409/4409 - 273s - loss: 0.7365 - mse: 0.6669 - val_loss: 1.0589 - val_mse: 0.9762
Epoch 3/3
4409/4409 - 272s - loss: 0.6281 - mse: 0.5378 - val_loss: 1.1167 - val_mse: 1.0160
MSE: 1.0871538


In [13]:
# Amazon Electronics

In [18]:
electronicstraindata = pd.read_csv("./df_electronics_train.csv") 

In [19]:
electronicstestdata = pd.read_csv("./df_electronics_test.csv") 

In [20]:
sparse_features = ['item_id','timestamp', 'user_id','brand', 'user_attr']

sequence_features = ['model_attr', 'category']

PK_features = ['item_id', 'user_id']

target = ['rating']

sep="&"

epochs = 3

batch_size = 126

In [23]:
model = RecommenderDeepFM(electronicstraindata,electronicstestdata, sparse_features, sequence_features, PK_features,
                                  target, batch_size, epochs, sep="&")

optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.5 | hidden_units:(128, 128) | epochs:3 | batch_size:32
Epoch 1/3


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


904/904 - 27s - loss: 1.8103 - mse: 1.8088 - val_loss: 1.3980 - val_mse: 1.3942
Epoch 2/3
904/904 - 29s - loss: 0.6482 - mse: 0.6421 - val_loss: 1.8083 - val_mse: 1.7998
Epoch 3/3
904/904 - 28s - loss: 0.3187 - mse: 0.3087 - val_loss: 1.4999 - val_mse: 1.4882
MSE: 1.5750498


In [33]:
df_modcloth_train=pd.read_csv("./df_modcloth_train.csv") 

In [34]:
df_modcloth_test = pd.read_csv("./df_modcloth_test.csv") 

In [35]:
sparse_features = ['item_id', 'user_id','timestamp','size','brand', 'user_attr','category']

sequence_features = ['model_attr']

PK_features = ['item_id', 'user_id']

target = ['rating']

epochs = 3

batch_size = 32

In [36]:
modclothmodel = RecommenderDeepFM(df_modcloth_train,df_modcloth_test, sparse_features, sequence_features, PK_features,
                                  target, batch_size, epochs, sep="&")

optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.5 | hidden_units:(128, 128) | epochs:3 | batch_size:32
Epoch 1/3


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


372/372 - 2s - loss: 1.8509 - mse: 1.8507 - val_loss: 1.0080 - val_mse: 1.0073
Epoch 2/3
372/372 - 2s - loss: 0.7217 - mse: 0.7203 - val_loss: 1.0922 - val_mse: 1.0901
Epoch 3/3
372/372 - 2s - loss: 0.4190 - mse: 0.4166 - val_loss: 1.1103 - val_mse: 1.1075
MSE: 1.2381465


In [21]:
tuningConfig = {"batch_size":[32,64],
               "epochs":[3,5],
               "embedding_dim":[32,128],
               "dnn_dropout":[0.2,0.5,0.8],
               "dnn_hidden_units":[(128,128),(256,128),(256,256)],
               "optimizer":["Adam","SGD"],
               "learning_rate":[0.0001,0.01,1]}

In [25]:
bestmodel={}
bestMSE=0.0
for optimizer in tuningConfig["optimizer"]:
    for learning_rate in tuningConfig["learning_rate"]:
        for dnn_hidden_units in tuningConfig["dnn_hidden_units"]:
            for dnn_dropout in tuningConfig["dnn_dropout"]:
                for embedding_dim in tuningConfig["embedding_dim"]:
                    for epochs in tuningConfig["epochs"]:
                        for batch_size in tuningConfig["batch_size"]:                            
                            model, MSE = RecommenderDeepFM(electronicstraindata,electronicstestdata, sparse_features, sequence_features, PK_features,
                                  target, batch_size, epochs, sep="&")
                            if MSE > bestMSE:
                                bestMSE = MSE
                                bestmodel = model

print('The best mse is {0}'.format(bestMSE))
# Save the model in order that we can use it as much as we want without training it everytime
model.save("DeepFMclothRating")

optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.5 | hidden_units:(128, 128) | epochs:3 | batch_size:32
Epoch 1/3


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


904/904 - 30s - loss: 1.8326 - mse: 1.8312 - val_loss: 1.4054 - val_mse: 1.4017
Epoch 2/3
904/904 - 26s - loss: 0.6526 - mse: 0.6466 - val_loss: 1.5633 - val_mse: 1.5548
Epoch 3/3
904/904 - 26s - loss: 0.3159 - mse: 0.3062 - val_loss: 1.4919 - val_mse: 1.4806
MSE: 1.5805844
optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.5 | hidden_units:(128, 128) | epochs:3 | batch_size:64
Epoch 1/3


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


452/452 - 12s - loss: 2.1239 - mse: 2.1231 - val_loss: 1.4089 - val_mse: 1.4067
Epoch 2/3
452/452 - 7s - loss: 0.6758 - mse: 0.6723 - val_loss: 1.4706 - val_mse: 1.4655
Epoch 3/3
452/452 - 9s - loss: 0.3143 - mse: 0.3084 - val_loss: 1.5368 - val_mse: 1.5300
MSE: 1.6379234
optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.5 | hidden_units:(128, 128) | epochs:5 | batch_size:32
Epoch 1/5


...

452/452 - 8s - loss: 2.1477 - mse: 2.1469 - val_loss: 1.3953 - val_mse: 1.3931
Epoch 2/3
452/452 - 8s - loss: 0.6730 - mse: 0.6694 - val_loss: 1.5151 - val_mse: 1.5098
Epoch 3/3
452/452 - 8s - loss: 0.2963 - mse: 0.2902 - val_loss: 1.5181 - val_mse: 1.5111
MSE: 1.5950853
optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.5 | hidden_units:(128, 128) | epochs:5 | batch_size:32
Epoch 1/5


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


904/904 - 60s - loss: 1.8007 - mse: 1.7992 - val_loss: 1.4393 - val_mse: 1.4354
Epoch 2/5
904/904 - 12s - loss: 0.6442 - mse: 0.6380 - val_loss: 1.5081 - val_mse: 1.4994
Epoch 3/5
904/904 - 12s - loss: 0.3181 - mse: 0.3080 - val_loss: 1.5073 - val_mse: 1.4955
Epoch 4/5
904/904 - 12s - loss: 0.1697 - mse: 0.1573 - val_loss: 1.5902 - val_mse: 1.5769
Epoch 5/5
904/904 - 12s - loss: 0.1372 - mse: 0.1232 - val_loss: 1.6381 - val_mse: 1.6232
MSE: 1.7054461
optimizer:Adam | embedding_dim:32 | lr:0.0001 | dropout:0.5 | hidden_units:(128, 128) | epochs:5 | batch_size:64
Epoch 1/5


C:\Users\Montr\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


452/452 - 6s - loss: 2.1196 - mse: 2.1188 - val_loss: 1.3893 - val_mse: 1.3871
Epoch 2/5
452/452 - 6s - loss: 0.6804 - mse: 0.6768 - val_loss: 1.5520 - val_mse: 1.5468
Epoch 3/5
452/452 - 7s - loss: 0.3031 - mse: 0.2971 - val_loss: 1.5312 - val_mse: 1.5243
Epoch 4/5
452/452 - 7s - loss: 0.1529 - mse: 0.1454 - val_loss: 1.6576 - val_mse: 1.6496
Epoch 5/5
452/452 - 7s - loss: 0.1052 - mse: 0.0969 - val_loss: 1.5751 - val_mse: 1.5665
MSE: 1.6548594
The best mse is 1.843369483947754
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: DeepFMclothRating\assets
